In [65]:
import copy
with open(r'Data\clustering1.txt','r') as f:
    edgelist=f.readlines()
edgelist = [(x.split()[0],x.split()[1],int(x.split()[2])) for x in edgelist[1:] ]


In [66]:
class KruskalCluster (object):
    def __init__(self, edgelist, k):
        #get the set of all nodes, content as node label
        self.nodeset=set([x[0] for x in edgelist]).union(set([x[1] for x in edgelist]))
        #nodedict stores the node label to node index mapping (node index is used to locate node leader node in union find stucture)
        self.nodedict={}
        index=0
        for node in self.nodeset:
            self.nodedict[node]=index
            index=index+1
        #clusterlist is the union find structure (underlying as an array), that stores the all nodes' leader node. 
        #index of the clusterlist is the node index, referenced in nodedict
        #content of the clusterlist is the node leader node, and node rank
        self.clusterlist= [None]*(len(self.nodedict)*2)
        for node in self.nodedict:
            #stores the leader node, initialize to itself
            self.clusterlist[self.nodedict[node]*2]=self.nodedict[node]
            #stores the node rank, intialize to 0
            self.clusterlist[self.nodedict[node]*2+1]=0
        #initialize with number of clusters
        self.k=k
        #initialize current number of clusters to number of node
        self.c=len(self.nodedict)
        #intialize and sort the edgelist
        self.edgelist=copy.deepcopy(edgelist)
        self.edgelist=sorted(self.edgelist, key=lambda x: x[2])
        #initialize the current minimum edge to the first one
        self.edgeProcessInd=0
    def find(self, node):
        #get the node's index
        currentnodeindex=self.nodedict[node]
        #find the node's parent node, until node = node parent node, means we found the root
        while self.clusterlist[currentnodeindex*2] != currentnodeindex:
            currentnodeindex=self.clusterlist[currentnodeindex*2]
        #path compression implementation, update the node's parent node directly to the root. Here ignore potential leader node rank change
        self.clusterlist[self.nodedict[node]*2]=currentnodeindex
        return currentnodeindex
    def union(self, node1, node2):
        node1leadernodeindex=self.find(node1)
        node2leadernodeindex=self.find(node2)
        #if node1's leader node rank >= node2's leader node rank, update node2's leader node's leader to node1's leader node
        if self.clusterlist[2*node1leadernodeindex+1] >= self.clusterlist[2*node2leadernodeindex+1]:
            self.clusterlist[2*node2leadernodeindex]=node1leadernodeindex
            #if node1's leader node rank = node2's leader node rank, update node1's leader node's rank to increase by 1
            if self.clusterlist[2*node1leadernodeindex+1] == self.clusterlist[2*node2leadernodeindex+1]:
                self.clusterlist[2*node1leadernodeindex+1] = self.clusterlist[2*node1leadernodeindex+1] +1
        #if node1's leader node rank < node2's leader node rank, update node1's leader node's leader to node2's leader node
        else:
            self.clusterlist[2*node1leadernodeindex]=node2leadernodeindex
    def cluster(self):
        for i in range(self.c, self.k, -1):
            mergedTwoCluster=False
            #while the current edge belongs to the same cluster, proceed to next edge, until the edge belongs to different cluster
            while mergedTwoCluster== False:
                #if the two vertices of the current edge belongs to two different cluster, then union the two clusters of the two vertices
                if self.find(self.edgelist[self.edgeProcessInd][0]) != self.find(self.edgelist[self.edgeProcessInd][1]):
                    self.union(self.edgelist[self.edgeProcessInd][0],self.edgelist[self.edgeProcessInd][1])
                    mergedTwoCluster=True
                    self.edgeProcessInd=self.edgeProcessInd+1
                else:
                    self.edgeProcessInd=self.edgeProcessInd+1
    def get_spacing(self):
        #find the next edge cost, whose vertices doesnt belong to the same cluster
        mergedTwoCluster=False
        #while the current edge belongs to the same cluster, proceed to next edge, until the edge belongs to different cluster
        while mergedTwoCluster== False:
            #if the two vertices of the current edge belongs to two different cluster
            if self.find(self.edgelist[self.edgeProcessInd][0]) != self.find(self.edgelist[self.edgeProcessInd][1]):
                mergedTwoCluster=True
                self.edgeProcessInd=self.edgeProcessInd+1
            else:
                self.edgeProcessInd=self.edgeProcessInd+1        
        return self.edgelist[self.edgeProcessInd-1][2]
            
        

In [67]:
kruskal=KruskalCluster(edgelist, 4)
kruskal.cluster()
kruskal.get_spacing()

90

In [ ]:
#Tested